**Upload train and test data**

In [ ]:
from google.colab import files #upload the data
train = files.upload()
test = files.upload()

**Import all the required packages**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import io
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

**Data Preprocessing**

Cleaing the training data

In [ ]:
train = pd.read_csv(io.BytesIO(train['train_indessa.csv']))

print(train.dtypes)
print("")
print(train.shape)
print("")
print(train['loan_status'].value_counts())
print("")
print(train.isnull().sum())

In [ ]:
half_count = len(train) / 2
t1 = train.dropna(thresh=half_count,axis=1) # Drop any column with more than 50% missing values
t1.isnull().sum()

In [ ]:
drop_list = ['member_id','funded_amnt','funded_amnt_inv','batch_enrolled','int_rate','total_rec_int','total_rec_late_fee','last_week_pay','collections_12_mths_ex_med','emp_length','tot_coll_amt','tot_cur_bal','total_rev_hi_lim','sub_grade','pymnt_plan','emp_title','zip_code','application_type','recoveries','collection_recovery_fee']
t1 = t1.drop(drop_list,axis=1) # Drop columns that give futuristic information or are irrelevant to the model or give redundant information
print(t1.isnull().sum())
print("")
print(t1.shape)
print("")
print(t1['loan_status'].value_counts())
print("")

In [ ]:
t1 = t1.dropna() # Drop rows that have missing values

print(t1.isnull().sum())
print("")
print(t1.shape)
print("")
print(t1['loan_status'].value_counts())
print("")
print(t1.dtypes)
print("")

Save the cleaned data to csv (if needed)

In [ ]:
t1.to_csv("filtered.csv",index=False)
files.download("filtered.csv")

Preparing train data for model training

In [ ]:
filtered_loans = t1.copy()

drop_cols = ['addr_state','title']
filtered_loans = filtered_loans.drop(drop_cols,axis=1)

mapping_dict = {
"grade":{
"A": 1,
"B": 2,
"C": 3,
"D": 4,
"E": 5,
"F": 6,
"G": 7
},
"term":{
    "36 months":1,
    "60 months":2
},
"verification_status":{
    "Source Verified":1,
    "Verified":1,
    "Not Verified":2
},
"initial_list_status":{
    "f":0,
    "w":1
}
}
filtered_loans = filtered_loans.replace(mapping_dict)
filtered_loans[['grade']].head()

nominal_columns = ["home_ownership", "purpose"]
dummy_df = pd.get_dummies(filtered_loans[nominal_columns])
filtered_loans = pd.concat([filtered_loans, dummy_df], axis=1)
filtered_loans = filtered_loans.drop(nominal_columns, axis=1)

print(filtered_loans.shape)
print("")
print(filtered_loans['loan_status'].value_counts())
print("")
print(filtered_loans.dtypes)

Save the final data for training

In [ ]:
filtered_loans.to_csv("train.csv",index=False)
files.download("train.csv")

Cleaning the test data

In [ ]:
test = pd.read_csv(io.BytesIO(test['test_indessa.csv']))

print(test.dtypes)
print("")
print(test.shape)
print("")
print(test.isnull().sum())

In [ ]:
drop_list = ['funded_amnt','funded_amnt_inv','batch_enrolled','int_rate','sub_grade','emp_title','pymnt_plan','desc','title','zip_code','addr_state','mths_since_last_delinq','mths_since_last_record','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','collections_12_mths_ex_med','mths_since_last_major_derog','application_type','verification_status_joint','last_week_pay','emp_length','tot_coll_amt','tot_cur_bal','total_rev_hi_lim']
test = test.drop(drop_list,axis=1)

test = test.dropna()

mapping_dict = {
"grade":{
"A": 1,
"B": 2,
"C": 3,
"D": 4,
"E": 5,
"F": 6,
"G": 7
},
"term":{
    "36 months":1,
    "60 months":2
},
"verification_status":{
    "Source Verified":1,
    "Verified":1,
    "Not Verified":2
},
"initial_list_status":{
    "f":0,
    "w":1
}
}
test = test.replace(mapping_dict)
test[['grade']].head()

nominal_columns = ["home_ownership", "purpose"]
dummy_df = pd.get_dummies(test[nominal_columns])
test = pd.concat([test, dummy_df], axis=1)
test = test.drop(nominal_columns, axis=1)

In [ ]:
idx = 16
new_col = np.empty(354736)
new_col.fill(0)
test.insert(loc=idx, column= "home_ownership_ANY", value=new_col)

print(test.head)

Save the test data

In [ ]:
test.to_csv("test.csv",index=False)
files.download("test.csv")

**Training the model**

In [ ]:
filtered_loans.rename(columns={"loan_status": "appstat"}, inplace=True)

f1 = filtered_loans.copy()

y = filtered_loans.appstat

filtered_loans.drop(columns = 'appstat', inplace=True)

X = filtered_loans.copy()

Over Sampling the data 

In [ ]:
smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(X, y)

#print('Original dataset shape', Counter(y))
#print('Resample dataset shape', Counter(y_ros))

print(X.shape)
print(x_smote.shape)
print(y.shape)
print(y_smote.shape)

Model

In [ ]:
x_train, x_cv, y_train, y_cv = train_test_split(x_smote,y_smote, test_size=0.2, random_state=4)

model = LogisticRegression()
model.fit(x_train, y_train)
#LogisticRegression()


print(roc_auc_score(y, model.predict_proba(X)[:, 1]))

print(roc_auc_score(y, model.decision_function(X)))

pred_cv = model.predict(x_cv)
print(accuracy_score(y_cv,pred_cv))

Test data prediction

In [ ]:
test.rename(columns={"member_id": "mid"}, inplace=True)

t2 = test.copy()

mid = test.mid

test.drop(columns = 'mid', inplace=True)

pred = model.predict(test)

for i in pred:
  print(i)

**Creating the output file**

In [ ]:
n1 = 'member_id'
n2 = 'loan_status'

data = {n1 : mid, n2 : pred}

df = pd.DataFrame(data)

df

In [ ]:
df.to_csv("output.csv",index=False)
files.download("output.csv")

In [ ]:
print(df['loan_status'].value_counts())